# <font color="Teal">Kerasによる深層学習の実践：画像編</font>

- version 0.1 (2019年)
    - 初版（教材作成：一関高専　小池 敦）
- version 0.2 (2022/1/16)
    - 2版（教材作成：一関高専　小池 敦）

本教材では，TensorFlowという深層学習プラットフォームを使用して深層学習を用いた画像処理を学びます．  
TensorFlowのハイレベルAPI（TensorFlowを簡単に使えるようにしたもの）であるKerasを使用し，以下のことを学びます．
* 画像ファイルを用いた深層学習
* 学習済みモデルを活用した深層学習

# 1. ✏️ <font color="Teal">準備</font>


## 1-1. <font color="Teal">教材の複製</font>

まず，本教材を自分のGoogleドライブに保存します．  
以下の手順を行ってください．


1.   <font color="OrangeRed">Chromeブラウザを利用</font>して，本教材にアクセスする．
2.   「ファイル」→<font color="OrangeRed">「ドライブにコピーを保存」</font>を選択し，自分のGoogleドライブに本教材の複製を作る．


教材中のコードを実行する際は，コードの左側にある実行ボタンを押します．


<font color="RoyalBlue">【実習】$3+5=8$ であることを確認する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
3 + 5
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

## 1-2. <font color="Teal">GPU利用</font>

本教材では計算時間を短くするためにGPUを利用します．
以下の手順を行ってください．


1.   Colabの「ランタイム」→<font color="OrangeRed">「ランタイムのタイプを変更」</font>で，ハードウェアアクセラレータを「None」から<font color="OrangeRed">「GPU」</font>に変更し保存する．  
（既に「GPU」になっている場合はそのままでよい．）


<font color="RoyalBlue">【実習】GPUが割り当てられていることを確認する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
!nvidia-smi
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

二重線の下の行に，GPUの名前（Tesra K80, Tesla T4 等）が表示されます．  
"NVIDIA-SMI has failed ・・・" と出力される場合は，上記手順が正しく行えていませんので，再度上記手順を行ってください．

# 2. ✏️ <font color="Teal">カラー写真の分類</font>

実際に画像ファイルを読み込んで画像分類を行います．  
ここでは，犬と猫の写真から，それらを分類するタスクを行います．

大量の画像ファイルを用いてモデルの学習を行う場合，あらかじめすべての画像ファイルをメモリに読み込もうとするとメモリが不足します．  
TensorFlowが提供する[tf.data](https://www.tensorflow.org/guide/data) APIを使用すると，そのような場合でも適切にデータ読み込みを行うことができます．  
また，tf.data APIでは，前処理やデータ拡張に関する各種機能も提供しています．

ここでは，tf.data APIを用いて画像ファイルを読み込み，モデルの学習を行います．

## 2-1. <font color="Teal">Dog vs Cat データ読み込み</font> 

まずは，TensorFlowとmatplotlibを読み込みます．

<font color="RoyalBlue">【実習】TensorFlowをインポートして，tf という別名を付けたのち，バージョンを確認する．NumPy，matplotlibも同時にインポートする．</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
print("TF version:", tf.__version__)
tf.random.set_seed(1234)
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```



次に[dogs_vs_cats](https://www.microsoft.com/en-us/download/details.aspx?id=54765)データセットで提供されている，様々なサイズの犬と猫の写真ファイルを読み込みます．  

`tf.keras.utils`の[`get_file`](https://www.tensorflow.org/api_docs/python/tf/keras/utils/get_file)関数を用いると，所定の場所にファイルをダウンロードできます．  
ダウンロードしたファイルは圧縮されていますが，引数として，`extract=True`と指定すると自動で解凍されます．


<font color="RoyalBlue">【実習】"/content"ディレクトリの中に"dogs_vs_cats"ディレクトリを作り，そこにdogs_vs_catsデータセットをダウンロードする．そして，ダウンロードした圧縮ファイルを同じ場所に解凍する．</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
#url = "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip"
url = "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip"
tf.keras.utils.get_file(
    origin = url,
    extract = True,
    cache_dir = "/content",
    cache_subdir = "dogs_vs_cats"
)
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


解凍されたデータのディレクトリ構成は以下のようになっており，猫の写真はCatディレクトリに，犬の写真はDogディレクトリに格納されています．

```
PetImages/
....Cat/
........0.jpg
........1.jpg
....Dog/
........0.jpg
........1.jpg
```


<font color="RoyalBlue">【実習】`PetImages`ディレクトリの中に`Cat`ディレクトリと`Dog`ディレクトリがあることを確認する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
!ls -la /content/dogs_vs_cats/PetImages
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


<font color="OrangeRed">(っ´･ω･)っ</font><font color="IndianRed">【Colabの画面からディレクトリ構成を確認することもできます．ページ左端上側にあるディレクトリのマークをクリックしましょう．
】</font>


## 2-2. <font color="Teal">画像ファイル一覧の取得</font> 

すべての画像ファイルのファイル名を管理するために，Pythonの[pathlib](https://docs.python.org/ja/3/library/pathlib.html)を利用します．  
pathlibを使うことで，ディレクトリ内のファイル一覧を取得したりすることが容易にできます．  
ここでは，pathlibの[`glob`](https://docs.python.org/ja/3/library/pathlib.html#pathlib.Path.glob)メソッドを用いて画像ファイル一覧を取得したのち，猫の画像ファイル名をリスト`cats[ ]`，犬の画像ファイル名をリスト`dogs[ ]`で管理することにします．

<font color="RoyalBlue">【実習】猫画像ファイルの一覧と犬画像ファイルの一覧を取得し，それぞれのファイル数を表示する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
import pathlib

dir_orig = pathlib.Path("/content/dogs_vs_cats/PetImages")
cats = list(dir_orig.glob('Cat/*.jpg'))
dogs = list(dir_orig.glob('Dog/*.jpg'))
print(len(cats), len(dogs))
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


<font color="RoyalBlue">【実習】猫画像ファイルを1枚表示する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
img = plt.imread(cats[0])
plt.imshow(img)
plt.show()
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


<font color="RoyalBlue">【実習】読み込んだ画像データのデータ型を確認する（NumPyアレイであることを確認する）</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
type(img)
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


<font color="RoyalBlue">【実習】読み込んだ画像データのデータサイズを確認する（縦，横，チャンネル数（RGBなら3））</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
img.shape
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


<font color="RoyalBlue">【実習】読み込んだ画像データの各画素値のデータ型を確認する（符号なし8ビット整数（0〜255）であることを確認する）</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
type(img[0, 0, 0])
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


<font color="RoyalBlue">【実習】読み込んだ画像データの画素値の最小値と最大値を確認する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
print(img.min(), img.max())
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


<font color="RoyalBlue">【実習】猫画像ファイルを12枚表示する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
plt.figure(figsize=(15, 10))
for i in range(12):
    ax = plt.subplot(3, 4, i + 1)
    img = plt.imread(cats[i])
    plt.imshow(img)
plt.show()
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


<font color="RoyalBlue">【実習】犬画像ファイルを12枚表示する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
plt.figure(figsize=(15, 10))
for i in range(12):
    ax = plt.subplot(3, 4, i + 1)
    img = plt.imread(dogs[i])
    plt.imshow(img)
plt.show()
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


このデータセットには壊れた画像ファイルが含まれています．  
[Kerasマニュアルのコード例](https://keras.io/examples/vision/image_classification_from_scratch/#filter-out-corrupted-images)を参考にして，そのようなファイルを除去します．

<font color="RoyalBlue">【実習】猫画像，犬画像のそれぞれについて，壊れたファイルを除去する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
import os

for folder_name in ("Cat", "Dog"):
    num_skipped = 0
    folder_path = os.path.join(dir_orig, folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)
    print("Deleted {} images in {}".format(num_skipped, folder_name))
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


## 2-3. <font color="Teal">学習データとテストデータの分割</font> 

ダウンロードしたデータは，下記のように，ラベルごとに異なるディレクトリに画像ファイルが格納されていました．

```
PetImages/
....Cat/
........0.jpg
........1.jpg
....Dog/
........0.jpg
........1.jpg
```

ここでは，このディレクトリ構成を残したまま，データを学習用，バリデーション用，テスト用に分割することを考えます．  
この処理を行うライブラリとして，[split-folders](https://pypi.org/project/split-folders/)があります．  
このライブラリを使用すると，データを指定した比率で3つに分割し，それぞれ`train`，`val`，`test`に格納することができます．データ分割後のディレクトリ構成は以下のようになります．

```
指定した出力ディレクトリ/
....train/
........Cat/
............xx.jpg
............xx.jpg
........Dog/
............xx.jpg
............xx.jpg
....val/
........Cat/
............xx.jpg
............xx.jpg
........Dog/
............xx.jpg
............xx.jpg
....test/
........Cat/
............xx.jpg
............xx.jpg
........Dog/
............xx.jpg
............xx.jpg
```


<font color="RoyalBlue">【実習】split-foldersライブラリをインストールする</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
!pip install split-folders
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


<font color="RoyalBlue">【実習】`splitfolders`をインポートする</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
import splitfolders
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


`splitfolders`の`ratio`関数を用いることで，データの分割を行い，分割後のデータを指定したディレクトリに格納することができます．
*   引数`output`で出力先を指定します
*   引数`seed`で分割に使用する乱数のシードを指定します
*   引数`ratio`で学習用，バリデーション用，テスト用のデータの割合を指定します
*   `move = False`とするとファイルを分割後のディレクトリに移動せず，コピーするようになります

<font color="RoyalBlue">【実習】`splitfolders`の`ratio`関数を用いて，データの分割を行い，分割後のデータを`/content/dogs_vs_cats/images`にコピーする
*   学習用，バリデーション用，テスト用の割合は，8:1:1 にする
</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
splitfolders.ratio(
    dir_orig,
    output = "/content/dogs_vs_cats/images",
    seed = 1234,
    ratio = (0.7, 0.15, 0.15),
    move = False
)
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


<font color="OrangeRed">(っ´･ω･)っ</font><font color="IndianRed">【間違った設定でデータを分割してしまった場合は，一旦コピー後のデータを削除しましょう．コードセルに以下のようなディレクトリ削除コマンドを打ち込むことで削除を行うことができます．

```
!rm -rf /content/dogs_vs_cats/images
```
】</font>


<font color="RoyalBlue">【実習】`images`ディレクトリの中に`train`ディレクトリ,
`val`ディレクトリ，`test`ディレクトリがあることを確認する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
!ls -la /content/dogs_vs_cats/images
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


## 2-4. <font color="Teal">データセット作成</font> 

### 2-4-1. <font color="Teal">`tf.data.Dataset`の概要</font> 

これまで，モデルの学習をする際は，（メモリ上の）配列に格納された画像データを1バッチ分の枚数ずつ読み込み，それを用いてパラメータ更新をしていました．  
しかし，この方式だと，すべてのデータをあらかじめメモリに置いておかなくてはならなくなります．画像データのサイズが大きい場合，すべてのデータをメモリに置くことができなくなり，この方法は使えません．

そこで，すべてのデータをあらかじめメモリ上に置くことはやめ，別の方法を考えます．  
具体的には，データセットオブジェクト（[`tf.data.Dataset`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset)）が学習中に動的に画像ファイルを読み込むことにし，直近で必要となる画像データのみをメモリに置くことにします．
そして，モデルからデータセットに対してデータを要求すると，データセットが1バッチ分のデータを返すようにします．
データセットオブジェクトは，今後必要となるデータを適宜ファイルから（GPU）メモリに読み込んでおき（プリフェッチと呼びます），モデルからの要求に応じてデータを渡します．

`tf.data`は上記の機能の他に，効率的な前処理実行をサポートする機能も備わっています．
`tf.data`の概要については，TensorFlowガイドの[データ入力パイプライン](https://www.tensorflow.org/guide/data)や，TensorFlowチュートリアルの[「tf.dataを使って画像をロードする」](https://www.tensorflow.org/tutorials/load_data/images?hl=ja)をご覧ください．

### 2-4-2. <font color="Teal">データセットオブジェクトの生成</font> 

画像ファイルからデータセットオブジェクトを作ります．  
以下のディレクトリ構成でファイルを格納しておくことで，入力データとラベルを同時に読み込むことができます．

* 1サンプル1ファイルとする（ファイル名はなんでも良い）
* クラスごとにディレクトリを作成し，ディレクトリ名をクラスのラベル名にする
* 各サンプルは対応するクラスのディレクトリに格納する

例えば，`Cat`と`Dog`のラベルを持つ画像データは以下のように格納します．
（split-foldersライブラリにより作成されたデータは以下の形式になっています）

```
メインディレクトリ/
....Cat/
........0.jpg
........1.jpg
....Dog/
........0.jpg
........1.jpg
```

上記のディレクトリ構成で格納されたデータからデータセット（`tf.data.Dataset`）を作るには`tf.keras.utils`の[`image_dataset_from_directory`](https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory)関数を使用します．  
引数の詳細については，[関数`image_dataset_from_directory`のドキュメント](https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory)をご覧ください．いくつかの注意事項について，下記にまとめます．


* 読み込んだデータはデフォルト設定ではシャッフルされます．引数`seed`でそれらに使用する乱数のシードを指定することができます．  
* 標準設定では，すべての画像は引数`image_size`で指定したサイズになるように縦と横を拡大縮小されます．縦横比（アスペクト比）は保存されません．
* データセット側でバッチサイズを指定します．指定には引数`batch_size`を使用します．
* 1つのデータから学習用データと検証用データに分割して読み込むこともできます（今回は既に分割済みなので使用しません）
* クラスのラベル名はディレクトリの名前になりますが，各クラスには0始まりの整数番号が付けられており，学習時にはその数字が使用されます．データセットオブジェクトが保持するリスト`class_names`に各番号に対応するラベル名が格納されています．例えば，データセット`ds`のクラス番号`1`に対応するラベル名は`ds.class_names[1]`に格納されています．


<font color="RoyalBlue">【実習】学習用データについてデータセット（`tf.data.Dataset`）を作る
*   画像サイズは，160 x 160 にする
*   バッチサイズは32にする（一度に32枚の画像を渡すようにする）
</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
dir_train = "/content/dogs_vs_cats/images/train"

seed = 1234
img_height = 160
img_width = 160
batch_size = 32

ds_train = tf.keras.utils.image_dataset_from_directory(
    dir_train,
    seed = seed,
    image_size = (img_height, img_width),
    batch_size = batch_size
)
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


<font color="RoyalBlue">【実習】データセットに格納されたデータのラベル名を確認する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
ds_train.class_names
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


<font color="RoyalBlue">【実習】バリデーション用データ，テスト用データについても学習用データと同様にデータセット（`tf.data.Dataset`）を作る
</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
dir_val = "/content/dogs_vs_cats/images/val"
dir_test = "/content/dogs_vs_cats/images/test"

ds_val = tf.keras.utils.image_dataset_from_directory(
    dir_val,
    seed = seed,
    image_size = (img_height, img_width),
    batch_size = batch_size
)

ds_test = tf.keras.utils.image_dataset_from_directory(
    dir_test,
    seed = seed,
    image_size = (img_height, img_width),
    batch_size = batch_size
)
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


### 2-4-3. <font color="Teal">データセットからのデータ取得</font> 

データセット（`tf.data.Dataset`）は[Pythonのイテラブルと呼ばれるオブジェクトになっています](https://www.tensorflow.org/guide/data?hl=en#basic_mechanics)．

Pythonの[イテラブル](https://docs.python.org/ja/3/glossary.html#term-iterable)は反復可能オブジェクトと呼ばれるもので，`for`文などの繰り返し処理において1要素ずつのアクセスが可能なオブジェクトのことです．イテラブルの例として，リストやタプルが挙げられます．  
`for`文においては，`for i in イテラブル:`のように書くことで，イテラブルの各要素にアクセスすることができます．

`for`文のイテラブル部分をデータセット（`tf.data.Dataset`）にすると，`for`ループの各反復において，バッチサイズ分のデータが返されます．  
前節のようにデータセットを作成した場合，各反復において返されるデータはバッチサイズ分の入力データとバッチサイズ分のラベルデータのペアとなっています．

<font color="OrangeRed">(っ´･ω･)っ</font><font color="IndianRed">【イテラブルに反復アクセスする別の方法として，[イテレータ](https://docs.python.org/ja/3/glossary.html#term-iterator)を使うやり方もあります．この方法では好きなタイミングで次のデータを読み込むことができます．`for`文においても内部でイテレータが生成されています．】</font>


まずは，1バッチのデータのデータ型について確認します．

<font color="RoyalBlue">【実習】学習用データセットから画像データとラベルを1バッチ分読み込み，それらのデータ型を確認する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
for images, labels in ds_train:
    print(type(images), type(labels))
    print(labels[0])
    break
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


<font color="OrangeRed">(っ´･ω･)っ</font><font color="IndianRed">【データセットから1バッチ分のデータを読み込むための別の書き方として，以下があります．
```
images, labels = next(iter(ds_train))
```
ここで，`iter(ds_train)`はイテレータを生成する処理で，`next()`はイテレータから1バッチ分のデータを取り出す処理です．】</font>


上記の実行結果より，データセットから返される入力データとラベルは共に`EagerTensor`型であることがわかります．  
これは，TensorFlowで定義されているTensorオブジェクトと呼ばれるものです．

Tensorオブジェクトが保持する属性値については，[tf.Tensorのドキュメント](https://www.tensorflow.org/api_docs/python/tf/Tensor#attributes_1)をご覧ください．  
`shape`については，NumPy配列と同様に使うことができます．

<font color="RoyalBlue">【実習】読み込んだ画像データとラベルのデータサイズを確認する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
for images, labels in ds_train:
    print(images.shape, labels.shape)
    break
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


次にTensorオブジェクトに格納されている各数値のデータ型を調べます．  
NumPy配列では，配列中の1要素を取り出して型を調べれば数値の型を調べることができましたが，Tensorオブジェクトにおいてはその方法は使えません．配列もそれを構成する各要素も両方Tensorオブジェクトになっているためです．

Tensorオブジェクトの各要素のデータ型を調べるには，Tensorオブジェクトの`dtype`属性を参照します．


<font color="RoyalBlue">【実習】読み込んだ画像データとラベルの各要素のデータ型を確認する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
for images, labels in ds_train:
    print(images.dtype, labels.dtype)
    break
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


画像データは元々8ビット符号なし整数型でしたが，浮動小数点型に変換されています．

<font color="RoyalBlue">【実習】学習用データセットから1バッチ分の画像データとラベルを読み込み表示する
</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
for images, labels in ds_train:
    print("Labels:", labels)
    plt.figure(figsize=(16, 8))
    for i in range(batch_size):
        ax = plt.subplot(4, 8, i + 1)
        plt.imshow(images[i]/255)
        plt.xticks([])
        plt.yticks([])
        plt.title(ds_train.class_names[labels[i]])
    plt.show()
    break
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


<font color="OrangeRed">(っ´･ω･)っ</font><font color="IndianRed">【画像を表示させる際，画像データの各要素を255で割っていますが，これはmatplotlibの`imshow()`関数の仕様のためです．`imshow()`関数では，[入力画像データの数値は，0-255の整数か，0-1の浮動小数点なくてはなりません](https://matplotlib.org/3.5.0/api/_as_gen/matplotlib.pyplot.imshow.html#matplotlib-pyplot-imshow)．`tf.data`のデータセットを作成すると，入力画像は浮動小数点型（float32型）に変換されてしまうため，入力画像の各要素は0-255の浮動小数点になってしまっています．そこで，255で割ることで，0-1の浮動小数点になるようにしています．】</font>


すべての画像データは縦と横を拡大縮小されることで同じサイズになっていることに注意しましょう．

## 2-5. <font color="Teal">入力画像の正規化</font> 

深層学習モデルの学習をしやすくするため，入力画像の画素値を0-255から0-1に正規化します．

ここでは，ファイルから画像を読み込む度に，画素値を1/255に変換することにします．  
データセットの[`map`メソッド](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map)を使用すると，データの問合せに対して，前処理を行った後のデータを返すような新しいデータセットオブジェクトを作成することができます．
1バッチ分のデータ（画像配列とラベル配列のペア）を受け取り，前処理後のデータを返すような関数を作っておき，それを`map`メソッドで指定します．

前処理関数を作成する際，Keras（TensorFlow）で用意された標準的な関数を活用することができます．  
画素値の変換を行う場合は，[`tf.keras.layers.Rescaling`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Rescaling)を使用して変換関数を作成することができます．


<font color="RoyalBlue">【実習】学習用データの問合せに対して，画像の画素値を1/255に変換したデータを返すような新しいデータセットを作成し，そのデータセットから得られる画像の画素値が0-1の範囲になっていること確認する．
* Tensorオブジェクトの全要素の最小値を得るには，[`tf.reduce_min()`](https://www.tensorflow.org/api_docs/python/tf/math/reduce_min)関数を使用する
* 同様に，最大値を得るには，[`tf.reduce_max()`](https://www.tensorflow.org/api_docs/python/tf/math/reduce_max)関数を使用する
</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
def preprocess_rescale(image, label):
    f = tf.keras.layers.Rescaling(1/255)
    return (f(image), label)

ds_train_norm = ds_train.map(preprocess_rescale)
for images, labels in ds_train_norm:
    print(tf.reduce_min(images), tf.reduce_max(images))
    break
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


<font color="OrangeRed">(っ´･ω･)っ</font><font color="IndianRed">【`map`メソッドに対して，前処理関数を渡す代わりに，[ラムダ式](https://docs.python.org/ja/3/reference/expressions.html?highlight=lambda#lambda)を渡しても構いません．その場合，新しいデータセットオブジェクトを作成する処理は，以下のような書き方になります（コロンの左側が引数，右側が戻り値となります）．
```
ds_train_norm = ds_train.map(lambda image, label: (tf.keras.layers.Rescaling(1/255)(image), label))
```
】</font>


<font color="RoyalBlue">【実習】バリデーション用データとテスト用データに対しても，前処理後のデータを返すようなデータセットを作成する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
ds_val_norm = ds_val.map(preprocess_rescale)
ds_test_norm = ds_test.map(preprocess_rescale)
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


## 2-6. <font color="Teal">データ読み込み速度の向上</font> 

データセットからのデータ読み込み速度を向上させるために2つの設定を行います．


*   [キャッシング](https://www.tensorflow.org/guide/data_performance#caching)：1回読み込んで前処理したデータをメモリもしくはストレージにキャッシュする（保持する）ことで，2回目以降の読み込み速度を速くします．データセットオブジェクトで[`cache()`メソッド](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#cache)を呼び出すことで，キャッシングを行うデータセットを生成します．
*   [プリフェッチ](https://www.tensorflow.org/guide/data_performance#prefetching)：モデルがあるバッチを処理している最中にそれ以降のバッチのデータを（GPU）メモリに転送します．プリフェッチする要素数については，バッファサイズを`tf.data.AUTOTUNE`とすることで，実行環境に応じた適切な値となります．データセットオブジェクトで[`prefetch()`メソッド](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#prefetch)を呼び出すことで，プリフェッチを行うデータセットを生成します．



<font color="RoyalBlue">【実習】学習用，バリデーション用，テスト用のそれぞれのデータセットに対して，キャッシングとプリフェッチを行うようにする</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
ds_train_norm = ds_train_norm.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
ds_val_norm = ds_val_norm.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
ds_test_norm = ds_test_norm.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


## 2-7. <font color="Teal">モデルの構築と学習</font>

畳み込みニューラルネットワークを作り，上記のデータセットで学習を行います．

<font color="RoyalBlue">【実習】簡易的な畳み込みニューラルネットワークを作りコンパイルする．そして作成したモデルを図示する．</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
tf.keras.utils.plot_model(model, show_shapes=True)
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


<font color="OrangeRed">(っ´･ω･)っ</font><font color="IndianRed">【モデルの出力については，2要素（猫，犬）にして，損失関数を`sparse_categorical_crossentropy`とすることもできますが，ここでは，1要素（0:猫，1:犬）にして，損失関数を`binary_crossentropy`にしています．
】</font>


<font color="RoyalBlue">【実習】作成したモデルの概要を表示する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
model.summary()
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


<font color="RoyalBlue">【実習】モデルを作成したデータセットで学習する（学習には学習用のデータセット，バリデーションにはバリデーション用のデータセットを使用する）</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
history = model.fit(
    ds_train_norm,
    validation_data=ds_val_norm,
    epochs=20,
    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)],
)
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


<font color="RoyalBlue">【実習】学習中の性能の変化を描画する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
plt.plot(history.history['accuracy'], 'o-', label='Training')
plt.plot(history.history['val_accuracy'], 'o-', label='Validation')
plt.legend()
plt.title('Accuracy')
plt.show()

plt.plot(history.history['loss'], 'o-', label='Training')
plt.plot(history.history['val_loss'], 'o-', label='Validation')
plt.legend()
plt.title('Loss')
plt.show()
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


## 2-8. <font color="Teal">モデルの評価</font>

<font color="RoyalBlue">【実習】テスト用データセットを用いて正解率を求める</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
test_loss, test_acc = model.evaluate(ds_test_norm)
test_acc
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


<font color="RoyalBlue">【実習】20サンプルについて，分類結果を表示する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
num = 5 * 4
plt.figure(figsize=(16, 20))
for images, labels in ds_test_norm:
    predictions = np.round(model.predict(images)).flatten().astype(int)
    for i in range(num):
        label = labels[i].numpy()
        plt.subplot(5, 4, i + 1)
        plt.imshow(images[i])
        plt.text(img_width - 50, img_height - 10, ds_train.class_names[label], color="cornflowerblue", size=24)
        plt.text(10, img_height - 10, ds_train.class_names[predictions[i]], color="red", size=24)
        if predictions[i] != label:
            plt.plot([1, img_width-8], [1, 1], color='red', linewidth=15)
        plt.xticks([])
        plt.yticks([])
    plt.show()
    break
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


<font color="OrangeRed">(っ´･ω･)っ</font><font color="IndianRed">【上記のコードについて補足します．モデルの出力は，0から1までの浮動小数点なので，[四捨五入](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.round.html?highlight=round#numpy.ndarray.round)し，整数に[型変換](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.astype.html?highlight=astype#numpy.ndarray.astype)することで，0（猫）か1（犬）に変換しています．また，1バッチ分の画像をモデルに入力すると，出力は，行数がバッチサイズ，列数が1の行列（2次元配列）になっているので，[`flatten()`関数](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.flatten.html)を呼ぶことで，1次元配列に変換しています．
】</font>


<font color="RoyalBlue">【実習】誤分類した20サンプルを表示する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
num = 5 * 4
plt.figure(figsize=(16, 20))
j = 0
for images, labels in ds_test_norm:
    predictions = np.round(model.predict(images)).flatten().astype(int)
    for i in range(num):
        label = labels[i].numpy()
        if predictions[i] != label:
            plt.subplot(5, 4, j + 1)
            plt.imshow(images[i])
            plt.text(img_width - 50, img_height - 10, ds_train.class_names[label], color="cornflowerblue", size=24)
            plt.text(10, img_height - 10, ds_train.class_names[predictions[i]], color="red", size=24)
            plt.plot([1, img_width-8], [1, 1], color='red', linewidth=15)
            plt.xticks([])
            plt.yticks([])
            j += 1
        if j >= num:
            break
    if j >= num:
        break
plt.show()
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


## 2-9. <font color="Teal">性能向上</font>

### 2-9-1. <font color="Teal">データ拡張</font>

過学習を軽減させる方法の一つにデータ数を増やすことがあります．  
学習時，より多くのサンプルを集められればいいのですが，それができない場合でも，既存の画像に，拡大や回転などの変換を行い，それを新しいサンプルとすることで，過学習を軽減させることができます．

画像の変換については，モデルに渡す前に行うこともできますが，ここでは，モデルの中で行うことにします．  
TensorFlowではこれらの変換処理を行うための層が用意されています．  
これらの層による変換処理は学習中のみ行われ，推論時には変換は行われません．

<font color="RoyalBlue">【実習】2.7節のネットワークの前段にデータ拡張を行う層を入れたネットワーク`model_new`を作り，コンパイルする．そして，作成したネットワークを図示する．
*   model_augmentation：データ拡張を行う層
    * RandomFlip：表裏を逆にする（`mode="horizontal"`を指定すると左右にひっくり返すが上下にはひっくり返さない）
    * RandomRotation：画像を回転させる（引数に`0.1`を指定すると，最大でプラスマイナス36度(=360*0.1)回転する）
    * RandomZoom：画像をランダムな位置でズームする（引数に`0.1`を指定すると，10%拡大する）
*   model_base：2.7節のネットワークの複製
</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
model_augmentation = tf.keras.models.Sequential([
    tf.keras.layers.RandomFlip(
        mode="horizontal",
        input_shape=(img_height, img_width, 3)
    ),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])
model_base = tf.keras.models.clone_model(model)
model_new = tf.keras.models.Sequential([
    model_augmentation,
    model_base
])
model_new.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
tf.keras.utils.plot_model(model_new, show_shapes=True)
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


<font color="RoyalBlue">【実習】作成したモデルの概要を表示する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
model_new.summary()
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


<font color="RoyalBlue">【実習】モデルを作成したデータセットで学習する（学習には学習用のデータセット，バリデーションにはバリデーション用のデータセットを使用する）</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
history = model_new.fit(
    ds_train_norm,
    validation_data=ds_val_norm,
    epochs=40,
    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)],
)
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


<font color="RoyalBlue">【実習】学習中の性能の変化を描画する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
plt.plot(history.history['accuracy'], 'o-', label='Training')
plt.plot(history.history['val_accuracy'], 'o-', label='Validation')
plt.legend()
plt.title('Accuracy')
plt.show()

plt.plot(history.history['loss'], 'o-', label='Training')
plt.plot(history.history['val_loss'], 'o-', label='Validation')
plt.legend()
plt.title('Loss')
plt.show()
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


<font color="RoyalBlue">【実習】テスト用データセットを用いて正解率を求める</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
test_loss, test_acc = model_new.evaluate(ds_test_norm)
test_acc
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


<font color="RoyalBlue">【実習】20サンプルについて，分類結果を表示する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
num = 5 * 4
plt.figure(figsize=(16, 20))
for images, labels in ds_test_norm:
    predictions = np.round(model_new.predict(images)).flatten().astype(int)
    for i in range(num):
        label = labels[i].numpy()
        plt.subplot(5, 4, i + 1)
        plt.imshow(images[i])
        plt.text(img_width - 50, img_height - 10, ds_train.class_names[label], color="cornflowerblue", size=24)
        plt.text(10, img_height - 10, ds_train.class_names[predictions[i]], color="red", size=24)
        if predictions[i] != label:
            plt.plot([1, img_width-8], [1, 1], color='red', linewidth=15)
        plt.xticks([])
        plt.yticks([])
    plt.show()
    break
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


<font color="RoyalBlue">【実習】誤分類した20サンプルを表示する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
num = 5 * 4
plt.figure(figsize=(16, 20))
j = 0
for images, labels in ds_test_norm:
    predictions = np.round(model_new.predict(images)).flatten().astype(int)
    for i in range(num):
        label = labels[i].numpy()
        if predictions[i] != label:
            plt.subplot(5, 4, j + 1)
            plt.imshow(images[i])
            plt.text(img_width - 50, img_height - 10, ds_train.class_names[label], color="cornflowerblue", size=24)
            plt.text(10, img_height - 10, ds_train.class_names[predictions[i]], color="red", size=24)
            plt.plot([1, img_width-8], [1, 1], color='red', linewidth=15)
            plt.xticks([])
            plt.yticks([])
            j += 1
        if j >= num:
            break
    if j >= num:
        break
plt.show()
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


### 2-9-2. <font color="Teal">転移学習</font>


既存の高性能ニューラルネットワーク活用して画像分類を行います．  
出力については利用シーンにより変わってくるので，
出力層はデータを使用して学習させます．

* メリット
    * 高性能な分類器を作りやすい
    * 学習のための時間を短縮できる（多くの重みが決定済み）
* 利用可能な画像分類モデル（ニューラルネットワーク）の一覧については[こちら](https://www.tensorflow.org/api_docs/python/tf/keras/applications#functions_2)
    * Xception, EfficientNetV2, MobileNetV2 などが有名
* 注意事項
    * 出力層は独自に学習させるため，既存ネットワークの出力層は取り除くようにする（`include_top=False`とする）
    * 既存ネットワークは今回は学習させない（重みを変えない）ようにする（`model.trainable=False`のように書く）
    * 既存ネットワークにおいて学習済みの重みを使用する場合は，どのデータセットを使用して学習した重みを使うのかを設定する（`weights="imagenet"`とするとImageNetデータセットで学習した重みが使用される）

<font color="RoyalBlue">【実習】Xceptionという学習済みモデルを活用して，モデル`model_trans`を作り，コンパイルする．そして，作成したネットワークを図示する．
* model_pre_trained：学習済みモデル（出力層は取り除く；学習させない）
* 学習済みモデルの出力に対し，チャンネルごとに全画素の平均値をとる（グローバルアベレージプーリングと呼ぶ）
* 上記の後に1層の全結合層を入れ，それを出力層とする
</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
model_pre_trained = tf.keras.applications.Xception(
    weights="imagenet",
    input_shape=(img_height, img_width, 3),
    include_top=False,
)
model_pre_trained.trainable = False
model_trans = tf.keras.models.Sequential([
    model_pre_trained,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model_trans.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
tf.keras.utils.plot_model(model_trans, show_shapes=True)
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


<font color="RoyalBlue">【実習】作成したモデルの概要を表示する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
model_trans.summary()
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


<font color="RoyalBlue">【実習】モデルを作成したデータセットで学習する（学習には学習用のデータセット，バリデーションにはバリデーション用のデータセットを使用する）</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
history = model_trans.fit(
    ds_train_norm,
    validation_data=ds_val_norm,
    epochs=40,
    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)],
)
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


<font color="RoyalBlue">【実習】学習中の性能の変化を描画する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
plt.plot(history.history['accuracy'], 'o-', label='Training')
plt.plot(history.history['val_accuracy'], 'o-', label='Validation')
plt.legend()
plt.title('Accuracy')
plt.show()

plt.plot(history.history['loss'], 'o-', label='Training')
plt.plot(history.history['val_loss'], 'o-', label='Validation')
plt.legend()
plt.title('Loss')
plt.show()
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


<font color="RoyalBlue">【実習】テスト用データセットを用いて正解率を求める</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
test_loss, test_acc = model_trans.evaluate(ds_test_norm)
test_acc
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


<font color="RoyalBlue">【実習】20サンプルについて，分類結果を表示する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
num = 5 * 4
plt.figure(figsize=(16, 20))
for images, labels in ds_test_norm:
    predictions = np.round(model_trans.predict(images)).flatten().astype(int)
    for i in range(num):
        label = labels[i].numpy()
        plt.subplot(5, 4, i + 1)
        plt.imshow(images[i])
        plt.text(img_width - 50, img_height - 10, ds_train.class_names[label], color="cornflowerblue", size=24)
        plt.text(10, img_height - 10, ds_train.class_names[predictions[i]], color="red", size=24)
        if predictions[i] != label:
            plt.plot([1, img_width-8], [1, 1], color='red', linewidth=15)
        plt.xticks([])
        plt.yticks([])
    plt.show()
    break
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


<font color="RoyalBlue">【実習】誤分類した20サンプルを表示する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
num = 5 * 4
plt.figure(figsize=(16, 20))
j = 0
for images, labels in ds_test_norm:
    predictions = np.round(model_trans.predict(images)).flatten().astype(int)
    for i in range(num):
        label = labels[i].numpy()
        if predictions[i] != label:
            plt.subplot(5, 4, j + 1)
            plt.imshow(images[i])
            plt.text(img_width - 50, img_height - 10, ds_train.class_names[label], color="cornflowerblue", size=24)
            plt.text(10, img_height - 10, ds_train.class_names[predictions[i]], color="red", size=24)
            plt.plot([1, img_width-8], [1, 1], color='red', linewidth=15)
            plt.xticks([])
            plt.yticks([])
            j += 1
        if j >= num:
            break
    if j >= num:
        break
plt.show()
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


## 2-10. <font color="Teal">演習</font>

* 自分で新しいネットワークを作って学習させてみる
* 変更案
    * コンパイル時に指定する最適化手法を変える（`optimizer='adam'`を`optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005)`に変える（標準のAdamから学習率0.0005のAdamに変える）等）
    * 学習（`fit`）時のエポック数（`epoch`）を大きくする
    * 2.9.2節のネットワークでXceptionを別のものに変える（[使用可能なモデル](https://www.tensorflow.org/api_docs/python/tf/keras/applications#functions_2)参照）